In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from qiskit.circuit.random import random_circuit

from purplecaffeine import LocalStorage, Trial, Widget

In [3]:
local_storage = LocalStorage("./trials")
for i in range(20):
    with Trial(f"My trial #{i}", storage=local_storage) as trial:
        trial.add_parameter("my parameter", i)
        for j in range(20):
            trial.add_metric("vqe_history", np.random.randint(0, 50))
          
        for j in range(20):
            trial.add_metric("other metric", np.random.randint(0, 50))
        
        for j in range(2):
            trial.add_circuit(f"circuit {j}", random_circuit(3, 2))

In [4]:
Widget(local_storage).show()

GridspecLayout(children=(Text(value='', description='Search:', layout=Layout(grid_area='widget001', width='95%…